In [2]:
import os
import h2o
import pandas as pd
import numpy as np
import datetime as dt
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator

In [3]:
h2o.cluster().shutdown()

AttributeError: 'NoneType' object has no attribute 'shutdown'

In [28]:
h2o.init(nthreads = -1, max_mem_size = 25)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_131"; OpenJDK Runtime Environment (build 1.8.0_131-8u131-b11-2ubuntu1.16.04.3-b11); OpenJDK 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from /home/laith/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0didmbcp
  JVM stdout: /tmp/tmp0didmbcp/h2o_laith_started_from_python.out
  JVM stderr: /tmp/tmp0didmbcp/h2o_laith_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_laith_wzp11j
H2O cluster total nodes:,1
H2O cluster free memory:,22.22 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [29]:
def prepare_data(with_validation = True):
    train = h2o.import_file(os.path.realpath("../data/train2.csv"))
    test  = h2o.import_file(os.path.realpath("../data/test2.csv"))
    
    x = train.columns
    y = "target"
    x.remove(y)
    
    test_id = test['id'].as_data_frame(True).id.values
    
    features_bin = [i for i in x if set(i)&set('bin')==set('bin')]
    features_cat = [i for i in x if (set(i)&set('cat')==set('cat'))&(set(i)&set('avg')!=set('avg'))]
    features_con = list(set(x) - (set(features_bin)|set(features_cat)))
    
    train[features_cat] = train[features_cat].asfactor()
    train[features_bin] = train[features_bin].asfactor()
    train[features_con] = train[features_con].asnumeric()
    
    test[features_cat] = test[features_cat].asfactor()
    test[features_bin] = test[features_bin].asfactor()
    test[features_con] = test[features_con].asnumeric()
    
    train['target'] = train['target'].asfactor()
    
    return train, test, x, y, test_id
train, test, x, y, test_id = prepare_data()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Tuning round 1

In [11]:
import math
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(22,27,1)),
                    'sample_rate': [x/100. for x in range(20,101)],
                    'col_sample_rate_per_tree': [x/100. for x in range(60,101)],
                    'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                    'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                    'nbins': [2**x for x in range(4,10)],
                    'nbins_cats': [2**x for x in range(8,13)],
                    'min_split_improvement': [0,1e-8,1e-6,1e-4],
                    'histogram_type': ["QuantilesGlobal","RoundRobin",'UniformAdaptive','Random']}

search_criteria_tune = {'strategy': "RandomDiscrete",
                       'max_runtime_secs': 60*60*8,  ## limit the runtime to 60 minutes
                       'max_models': 200,  ## build no more than 100 models
                       'seed' : 1234,
                       'stopping_rounds' : 5,
                       'stopping_metric' : "AUC",
                       'stopping_tolerance': 1e-3
                       }

my_rdf= H2ORandomForestEstimator(binomial_double_trees=True,
                                 nfolds=nfolds,
                                 fold_assignment="Stratified",
                                 keep_cross_validation_predictions=True,
                                 stopping_rounds = 5,
                                 stopping_metric = "AUC",
                                 stopping_tolerance = 1e-4,
                                 seed=1)

In [12]:
final_grid = H2OGridSearch(my_rdf, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune)
final_grid.train(x=x, y=y, training_frame=train)

drf Grid Build progress: |████████████████████████████████████████████████| 100%
Errors/Warnings building gridsearch model

Hyper-parameter: col_sample_rate_change_per_level, 0.91
Hyper-parameter: col_sample_rate_per_tree, 1.0
Hyper-parameter: histogram_type, Random
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 262144.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 2048
Hyper-parameter: sample_rate, 0.61
failure_details: Illegal argument(s) for DRF model: final_grid_model_12.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476591.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476339.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) 


Hyper-parameter: col_sample_rate_change_per_level, 1.09
Hyper-parameter: col_sample_rate_per_tree, 0.73
Hyper-parameter: histogram_type, Random
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 2.0
Hyper-parameter: min_split_improvement, 0.0001
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 1024
Hyper-parameter: sample_rate, 0.98
failure_details: Illegal argument(s) for DRF model: final_grid_model_62_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (355.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_62_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (355.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgument


Hyper-parameter: col_sample_rate_change_per_level, 0.93
Hyper-parameter: col_sample_rate_per_tree, 0.63
Hyper-parameter: histogram_type, QuantilesGlobal
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 2.0
Hyper-parameter: min_split_improvement, 1e-08
Hyper-parameter: nbins, 128
Hyper-parameter: nbins_cats, 256
Hyper-parameter: sample_rate, 0.35
failure_details: Illegal argument(s) for DRF model: final_grid_model_67_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (166.7 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_67_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (166.7 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegal


Hyper-parameter: col_sample_rate_change_per_level, 1.05
Hyper-parameter: col_sample_rate_per_tree, 0.87
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 22
Hyper-parameter: min_rows, 4.0
Hyper-parameter: min_split_improvement, 0.0001
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 256
Hyper-parameter: sample_rate, 0.64
failure_details: Illegal argument(s) for DRF model: final_grid_model_72_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (224.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_72_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (224.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgum


Hyper-parameter: col_sample_rate_change_per_level, 1.02
Hyper-parameter: col_sample_rate_per_tree, 0.89
Hyper-parameter: histogram_type, Random
Hyper-parameter: max_depth, 26
Hyper-parameter: min_rows, 8192.0
Hyper-parameter: min_split_improvement, 0.0
Hyper-parameter: nbins, 128
Hyper-parameter: nbins_cats, 512
Hyper-parameter: sample_rate, 0.3
failure_details: Illegal argument(s) for DRF model: final_grid_model_77_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 553  B per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_77_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 553  B per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentExc

...


Hyper-parameter: col_sample_rate_change_per_level, 1.04
Hyper-parameter: col_sample_rate_per_tree, 0.71
Hyper-parameter: histogram_type, Random
Hyper-parameter: max_depth, 24
Hyper-parameter: min_rows, 64.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 64
Hyper-parameter: nbins_cats, 2048
Hyper-parameter: sample_rate, 0.76
failure_details: Illegal argument(s) for DRF model: final_grid_model_175_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (78.5 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_175_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (78.5 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgument


Hyper-parameter: col_sample_rate_change_per_level, 0.95
Hyper-parameter: col_sample_rate_per_tree, 0.75
Hyper-parameter: histogram_type, QuantilesGlobal
Hyper-parameter: max_depth, 22
Hyper-parameter: min_rows, 16.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 512
Hyper-parameter: nbins_cats, 1024
Hyper-parameter: sample_rate, 0.82
failure_details: Illegal argument(s) for DRF model: final_grid_model_180_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (169.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_180_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (169.0 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIll


Hyper-parameter: col_sample_rate_change_per_level, 1.0
Hyper-parameter: col_sample_rate_per_tree, 0.77
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 25
Hyper-parameter: min_rows, 512.0
Hyper-parameter: min_split_improvement, 1e-08
Hyper-parameter: nbins, 256
Hyper-parameter: nbins_cats, 256
Hyper-parameter: sample_rate, 0.69
failure_details: Illegal argument(s) for DRF model: final_grid_model_184_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (12.2 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_184_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (12.2 KB per tree x 50 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgu


Hyper-parameter: col_sample_rate_change_per_level, 1.01
Hyper-parameter: col_sample_rate_per_tree, 0.65
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 25
Hyper-parameter: min_rows, 262144.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 256
Hyper-parameter: sample_rate, 0.99
failure_details: Illegal argument(s) for DRF model: final_grid_model_189.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476591.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476339.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476265.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: 


Hyper-parameter: col_sample_rate_change_per_level, 1.05
Hyper-parameter: col_sample_rate_per_tree, 0.6
Hyper-parameter: histogram_type, Random
Hyper-parameter: max_depth, 24
Hyper-parameter: min_rows, 262144.0
Hyper-parameter: min_split_improvement, 0.0
Hyper-parameter: nbins, 32
Hyper-parameter: nbins_cats, 1024
Hyper-parameter: sample_rate, 0.88
failure_details: Illegal argument(s) for DRF model: final_grid_model_197.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476591.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476339.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must have at least 524288.0 (weighted) rows, but have only 476265.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=262144.0: must h

In [13]:
sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)

In [20]:
sorted_final_grid

     col_sample_rate_change_per_level col_sample_rate_per_tree  \
0                                0.92                      0.9   
1                                0.99                      1.0   
2                                1.04                     0.95   
3                                0.99                     0.86   
4                                1.02                     0.79   
5                                1.05                     0.97   
6                                1.08                     0.63   
7                                0.93                     0.71   
8                                0.94                     0.65   
9                                1.07                      1.0   
10                               1.05                     0.88   
11                                0.9                     0.79   
12                               1.01                     0.99   
13                                0.9                     0.93   
14        

58       1024        0.73  final_grid_model_45  0.5884654145219325  


In [15]:
best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])

In [19]:
best_model.params

{'balance_classes': {'actual': False, 'default': False},
 'binomial_double_trees': {'actual': True, 'default': False},
 'build_tree_one_node': {'actual': False, 'default': False},
 'calibrate_model': {'actual': False, 'default': False},
 'calibration_frame': {'actual': None, 'default': None},
 'categorical_encoding': {'actual': 'AUTO', 'default': 'AUTO'},
 'checkpoint': {'actual': None, 'default': None},
 'class_sampling_factors': {'actual': None, 'default': None},
 'col_sample_rate_change_per_level': {'actual': 0.92, 'default': 1.0},
 'col_sample_rate_per_tree': {'actual': 0.9, 'default': 1.0},
 'fold_assignment': {'actual': 'Stratified', 'default': 'AUTO'},
 'fold_column': {'actual': None, 'default': None},
 'histogram_type': {'actual': 'RoundRobin', 'default': 'AUTO'},
 'ignore_const_cols': {'actual': True, 'default': True},
 'ignored_columns': {'actual': None, 'default': None},
 'keep_cross_validation_fold_assignment': {'actual': False, 'default': False},
 'keep_cross_validation_pr

### Tuning round 2

In [39]:
import math
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune2 = {#'calibrate_model':[True, False],
                      #'sample_rate_per_class':[x/10. for x in range(7,10)],
                      #'nbins_top_level': [2**x for x in range(8,13)],
                      'ntrees':list(range(50,800,50)),
                      'max_depth' : list(range(22,27,1))
                      #'balance_classes':[True, False]
                     }

search_criteria_tune2 = {'strategy': "RandomDiscrete",
                       'max_runtime_secs': 60*60*1.5,  ## limit the runtime to 60 minutes
                       'max_models': 200,  ## build no more than 100 models
                       'seed' : 1234,
                       'stopping_rounds' : 5,
                       'stopping_metric' : "AUC",
                       'stopping_tolerance': 1e-3
                       }

my_rdf2= H2ORandomForestEstimator(#max_depth = 23,
                                 sample_rate = 0.99,
                                 col_sample_rate_per_tree = 0.9,
                                 col_sample_rate_change_per_level = 0.92,
                                 #min_rows = 256, 
                                 nbins = 32, 
                                 nbins_cats = 1024, 
                                 min_split_improvement = 0,
                                 histogram_type = 'RoundRobin',
                                 binomial_double_trees=True,
                                 nfolds=nfolds,
                                 fold_assignment="Stratified",
                                 keep_cross_validation_predictions=True,
                                 stopping_rounds = 5,
                                 stopping_metric = "auc",
                                 stopping_tolerance = 1e-4,
                                 seed=1)

final_grid2 = H2OGridSearch(my_rdf2, hyper_params = hyper_params_tune2,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune2)

final_grid2.train(x=x, y=y, training_frame=train)

drf Grid Build progress: |████████████████████████████████████████████████| 100%
Errors/Warnings building gridsearch model

Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 1024
Hyper-parameter: ntrees, 700
Hyper-parameter: sample_rate_per_class, [0.5]
failure_details: Illegal argument(s) for DRF model: final_grid_model_0.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuild


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 1024
Hyper-parameter: ntrees, 450
Hyper-parameter: sample_rate_per_class, [0.4]
failure_details: Illegal argument(s) for DRF model: final_grid_model_4.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_4.  Details: ERRR on field: _sample_rate_pe


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 2048
Hyper-parameter: ntrees, 650
Hyper-parameter: sample_rate_per_class, [0.8]
failure_details: Illegal argument(s) for DRF model: final_grid_model_8.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_8.  Details: ERRR on field: _sample_rate_pe


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 4096
Hyper-parameter: ntrees, 700
Hyper-parameter: sample_rate_per_class, [0.3]
failure_details: Illegal argument(s) for DRF model: final_grid_model_12.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_12.  Details: ERRR on field: _sample_rate_


Hyper-parameter: balance_classes, False
Hyper-parameter: nbins_top_level, 4096
Hyper-parameter: ntrees, 500
Hyper-parameter: sample_rate_per_class, [0.7]
failure_details: Illegal argument(s) for DRF model: final_grid_model_16.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_16.  Details: ERRR on field: _sample_rate

...


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 1024
Hyper-parameter: ntrees, 600
Hyper-parameter: sample_rate_per_class, [0.9]
failure_details: Illegal argument(s) for DRF model: final_grid_model_84.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_84.  Details: ERRR on field: _sample_rate_


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 256
Hyper-parameter: ntrees, 300
Hyper-parameter: sample_rate_per_class, [0.8]
failure_details: Illegal argument(s) for DRF model: final_grid_model_88.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_88.  Details: ERRR on field: _sample_rate_p


Hyper-parameter: balance_classes, True
Hyper-parameter: nbins_top_level, 1024
Hyper-parameter: ntrees, 500
Hyper-parameter: sample_rate_per_class, [0.8]
failure_details: Illegal argument(s) for DRF model: final_grid_model_92.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_92.  Details: ERRR on field: _sample_rate_


Hyper-parameter: balance_classes, False
Hyper-parameter: nbins_top_level, 256
Hyper-parameter: ntrees, 100
Hyper-parameter: sample_rate_per_class, [0.9]
failure_details: Illegal argument(s) for DRF model: final_grid_model_96.  Details: ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).
ERRR on field: _sample_rate_per_class: _sample_rate_per_class must have 2 values (one per class).

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_96.  Details: ERRR on field: _sample_rate_

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.


In [40]:
sorted_final_grid2 = final_grid2.get_grid(sort_by='auc',decreasing=True)

In [41]:
sorted_final_grid2

    balance_classes nbins_top_level ntrees sample_rate_per_class  \
0             false            1024    400                  None   
1             false            2048     92                  None   
2             false            1024     50                  None   
3             false            1024     40                  None   
4              true            2048    111                  None   

            model_ids                 auc  
0  final_grid_model_2  0.6284895037150812  
1  final_grid_model_1    0.62111200797506  
2  final_grid_model_3  0.6153436801127311  
3  final_grid_model_4  0.6119871634066725  
4  final_grid_model_0  0.5480383808263608  


### final version

In [50]:
optimalRF= H2ORandomForestEstimator(ntrees = 100,
                                    max_depth = 23,
                                    sample_rate = 0.8,
                                    col_sample_rate_per_tree = 0.9,
                                    col_sample_rate_change_per_level = 0.92,
                                    min_rows = 256, 
                                    nbins = 32, 
                                    nbins_cats = 1024, 
                                    min_split_improvement = 0,
                                    histogram_type = 'RoundRobin',
                                    binomial_double_trees=True,
                                    nfolds=5,
                                    fold_assignment="Modulo",
                                    keep_cross_validation_predictions=True,
                                    stopping_rounds = 5,
                                    stopping_metric = "auc",
                                    stopping_tolerance = 1e-4,
                                    seed=1)

optimalRF.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [52]:
optimalRF.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.8688568,0.009767219,0.8688961,0.8940131,0.85700846,0.8689538,0.85541236
1,auc,0.63458216,0.0032236713,0.6385976,0.6405375,0.633845,0.63210523,0.6278255
2,err,0.13114323,0.009767219,0.13110389,0.105986916,0.14299154,0.13104619,0.14458762
3,err_count,15611.6,1162.682,15607.0,12617.0,17022.0,15600.0,17212.0
4,f0point5,0.087821476,0.0029263587,0.08798849,0.094784044,0.085586645,0.08854874,0.08219946
5,f1,0.11411791,0.0018711867,0.11509894,0.11713666,0.113714464,0.11534536,0.10929414
6,f2,0.16349068,0.0038845418,0.16635527,0.15328547,0.16938111,0.16539812,0.16303341
7,lift_top_group,3.3969104,0.23316729,3.6346233,3.9178843,3.2220082,3.019124,3.190912
8,logloss,0.15245427,6.3371594E-4,0.15096512,0.1520422,0.15257941,0.15347582,0.15320884
9,max_per_class_error,0.768743,0.014246079,0.76340324,0.80700946,0.7485609,0.7672769,0.7574644


In [49]:
optimalRF.cross_validation_predictions()

predict,p0,p1
0,0,0
0,0,0
0,0,0
0,0,0
0,0.961102,0.0388975
0,0,0
0,0,0
0,0,0
0,0,0
0,0,0


predict,p0,p1
0,0,0
0,0,0
0,0,0
0,0.982326,0.017674
0,0,0
0,0.969838,0.0301616
0,0,0
0,0,0
0,0,0
0,0,0


predict,p0,p1
0,0,0
0,0,0
0,0.97508,0.0249205
0,0,0
0,0,0
0,0,0
0,0,0
0,0,0
0,0.967946,0.0320537
0,0,0


predict,p0,p1
0,0.947931,0.0520694
0,0,0
0,0,0
0,0,0
0,0,0
0,0,0
0,0,0
0,0.974994,0.0250063
0,0,0
0,0,0


predict,p0,p1
0,0,0
0,0.968505,0.0314952
0,0,0
0,0,0
0,0,0
0,0,0
0,0.974212,0.0257878
0,0,0
0,0,0
1,0.939129,0.0608712


[, , , , ]

In [ ]:
sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)

In [ ]:
best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])